In [83]:
import json
import os
import csv
import numpy as np
import pandas as pd
import requests
from azureml.core.webservice import Webservice
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import sys
import joblib
from azureml.core import Datastore, Dataset, Run
from azureml.core import Workspace, Datastore

In [84]:
#Obtenemos los datos
ws = Workspace.from_config()

dataset = Dataset.get_by_name(ws, name='iris.csv')
df = dataset.to_pandas_dataframe()
df = df.sample(frac=0.2, replace=False, random_state=1)

print("Tamaño de la muestra:", len(df))
df.head()

Tamaño de la muestra: 30


,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
14,5.8,4.0,1.2,0.2,Iris-setosa
98,5.1,2.5,3.0,1.1,Iris-versicolor
75,6.6,3.0,4.4,1.4,Iris-versicolor
16,5.4,3.9,1.3,0.4,Iris-setosa
131,7.9,3.8,6.4,2.0,Iris-virginica


In [85]:
#Eliminamos la columna "species"
x_valid = df.drop(["Species"],axis=1)
x_valid.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
14,5.8,4.0,1.2,0.2
98,5.1,2.5,3.0,1.1
75,6.6,3.0,4.4,1.4
16,5.4,3.9,1.3,0.4
131,7.9,3.8,6.4,2.0


In [86]:
#Pasamos la muestra de datos a formato json
data = json.dumps({'data': x_valid.values.tolist()})
print(data)

{"data": [[5.8, 4.0, 1.2, 0.2], [5.1, 2.5, 3.0, 1.1], [6.6, 3.0, 4.4, 1.4], [5.4, 3.9, 1.3, 0.4], [7.9, 3.8, 6.4, 2.0], [6.3, 3.3, 4.7, 1.6], [6.9, 3.1, 5.1, 2.3], [5.1, 3.8, 1.9, 0.4], [4.7, 3.2, 1.6, 0.2], [6.9, 3.2, 5.7, 2.3], [5.6, 2.7, 4.2, 1.3], [5.4, 3.9, 1.7, 0.4], [7.1, 3.0, 5.9, 2.1], [6.4, 3.2, 4.5, 1.5], [6.0, 2.9, 4.5, 1.5], [4.4, 3.2, 1.3, 0.2], [5.8, 2.6, 4.0, 1.2], [5.6, 3.0, 4.5, 1.5], [5.4, 3.4, 1.5, 0.4], [5.0, 3.2, 1.2, 0.2], [5.5, 2.6, 4.4, 1.2], [5.4, 3.0, 4.5, 1.5], [6.7, 3.0, 5.0, 1.7], [5.0, 3.5, 1.3, 0.3], [7.2, 3.2, 6.0, 1.8], [5.7, 2.8, 4.1, 1.3], [5.5, 4.2, 1.4, 0.2], [5.1, 3.8, 1.5, 0.3], [6.1, 2.8, 4.7, 1.2], [6.3, 2.5, 5.0, 1.9]]}


In [87]:
#URL para consumir el modelo
URI = "ACI_CONTAINER_SERVICE_URI"
headers = {'Content-Type': 'application/json'}

response = requests.post(URI, data=data, headers=headers)
                                                                
assert response.status_code == 200

In [88]:
y_pred = np.array(response.json())
y_pred

array(['Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa',
       'Iris-virginica', 'Iris-versicolor', 'Iris-virginica',
       'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor',
       'Iris-setosa', 'Iris-virginica', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-virginica', 'Iris-setosa',
       'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa',
       'Iris-versicolor', 'Iris-virginica'], dtype='<U15')

In [89]:
#Seleccionamos la columna con las etiquetas
y_valid = df["Species"]
y_valid.head()

14         Iris-setosa
98     Iris-versicolor
75     Iris-versicolor
16         Iris-setosa
131     Iris-virginica
Name: Species, dtype: object

In [90]:
#Calculamos el porcentaje de precisión de nuestra muestra
accuracy_score(y_valid, y_pred)

0.9666666666666667

In [91]:
#Analizamos la matriz de confusión
confusion_matrix(y_valid, y_pred, labels=["Iris-setosa", "Iris-versicolor", "Iris-virginica"])

array([[11,  0,  0],
       [ 0, 12,  1],
       [ 0,  0,  6]])